# Koosta regressioonimudel: lineaar- ja polünoomregressioonimudelid


## Lineaarne ja polünoomne regressioon kõrvitsate hinnastamiseks - Õppetund 3
<p >
   <img src="../../../../../../translated_images/et/linear-polynomial.5523c7cb6576ccab.png"
   width="800"/>
   <figcaption>Infograafika autor: Dasani Madipalli</figcaption>


#### Sissejuhatus

Siiani olete uurinud, mis on regressioon, kasutades näidisandmeid kõrvitsate hinnastamise andmestikust, mida kasutame kogu selle õppetunni vältel. Olete seda ka visualiseerinud, kasutades `ggplot2`.💪

Nüüd olete valmis süvenema regressiooni kasutamisse masinõppes. Selles õppetunnis õpite rohkem kahte tüüpi regressiooni kohta: *lihtne lineaarne regressioon* ja *polünoomne regressioon*, koos mõningate matemaatiliste alustega, mis neid tehnikaid toetavad.

> Kogu selle õppekava jooksul eeldame minimaalset matemaatikaalast teadmist ja püüame muuta selle arusaadavaks teiste valdkondade õppijatele. Jälgige märkmeid, 🧮 selgitusi, diagramme ja teisi õppimist toetavaid vahendeid.

#### Ettevalmistus

Tuletame meelde, et laadite need andmed, et neilt küsimusi küsida.

-   Millal on parim aeg kõrvitsate ostmiseks?

-   Millist hinda võin oodata miniatuursete kõrvitsate kasti eest?

-   Kas peaksin ostma neid poole korvi kaupa või 1 1/9 korvi kasti kaupa? Uurime seda andmestikku edasi.

Eelmises õppetunnis lõite `tibble` (kaasaegne andmeraami uuendus) ja täitsite selle osa algsest andmestikust, standardiseerides hinnad korvi järgi. Sellega suutsime koguda umbes 400 andmepunkti ja ainult sügiskuude kohta. Võib-olla saame andmete olemuse kohta rohkem teada, kui neid veelgi puhastame? Eks näeme... 🕵️‍♀️

Selle ülesande jaoks vajame järgmisi pakette:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) on [R-pakettide kogum](https://www.tidyverse.org/packages), mis on loodud selleks, et muuta andmeteadus kiiremaks, lihtsamaks ja lõbusamaks!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) raamistik on [pakettide kogum](https://www.tidymodels.org/packages/) modelleerimiseks ja masinõppeks.

-   `janitor`: [janitor pakett](https://github.com/sfirke/janitor) pakub lihtsaid tööriistu määrdunud andmete uurimiseks ja puhastamiseks.

-   `corrplot`: [corrplot pakett](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) pakub visuaalset uurimisvahendit korrelatsioonimaatriksile, mis toetab automaatset muutujate ümberjärjestamist, et aidata tuvastada peidetud mustreid muutujate vahel.

Saate need installida järgmiselt:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Allolev skript kontrollib, kas teil on selle mooduli lõpetamiseks vajalikud paketid olemas, ja installib need vajadusel.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Hiljem laadime need suurepärased paketid ja teeme need meie praeguses R-sessioonis kättesaadavaks. (See on lihtsalt illustratsiooniks, `pacman::p_load()` tegi selle juba teie eest ära)

## 1. Lineaarse regressiooni joon

Nagu õppisite 1. õppetunnis, on lineaarse regressiooni eesmärk olla võimeline joonistama *parima sobivuse joont*, et:

-   **Näidata muutujate seoseid**. Näidata muutujate omavahelist seost.

-   **Teha prognoose**. Teha täpseid prognoose selle kohta, kuhu uus andmepunkt selle joone suhtes paigutuks.

Selle tüüpi joone joonistamiseks kasutame statistilist tehnikat, mida nimetatakse **väikseimate ruutude regressiooniks**. Termin `väikseimad ruudud` tähendab, et kõik regressioonijoone ümber olevad andmepunktid ruudustatakse ja seejärel liidetakse. Ideaalis on lõplik summa võimalikult väike, sest me soovime väikest vigade arvu ehk `väikseimad ruudud`. Seega on parima sobivuse joon see joon, mis annab meile väikseima ruutude vigade summa - sellest ka nimi *väikseimate ruutude regressioon*.

Teeme seda, kuna soovime modelleerida joont, millel on kõige väiksem kumulatiivne kaugus kõigist meie andmepunktidest. Samuti ruudustame terminid enne nende liitmist, kuna meid huvitab nende suurus, mitte suund.

> **🧮 Näita mulle matemaatikat**
>
> Seda joont, mida nimetatakse *parima sobivuse jooneks*, saab väljendada [võrrandiga](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` on '`selgitav muutuja` või `ennustaja`'. `Y` on '`sõltuv muutuja` või `tulemus`'. Joone kalle on `b` ja `a` on y-teljel lõikepunkt, mis viitab `Y` väärtusele, kui `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "kalle = $y/x$")
    Infograafika: Jen Looper
>
> Kõigepealt arvutage kalle `b`.
>
> Teisisõnu, viidates meie kõrvitsate andmete algsele küsimusele: "prognoosige kõrvitsa hind kuus müügi kuu järgi", oleks `X` hind ja `Y` müügi kuu.
>
> ![](../../../../../../translated_images/et/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.png)
    Infograafika: Jen Looper
> 
> Arvutage Y väärtus. Kui maksate umbes 4 dollarit, peab olema aprill!
>
> Matemaatika, mis arvutab joone, peab näitama joone kallet, mis sõltub ka lõikepunktist ehk sellest, kus `Y` asub, kui `X = 0`.
>
> Saate vaadata nende väärtuste arvutamise meetodit [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html) veebisaidil. Külastage ka [väikseimate ruutude kalkulaatorit](https://www.mathsisfun.com/data/least-squares-calculator.html), et näha, kuidas numbrite väärtused joont mõjutavad.

Polegi nii hirmutav, eks? 🤓

#### Korrelatsioon

Veel üks termin, mida mõista, on **korrelatsioonikordaja** antud X ja Y muutujate vahel. Hajuvusdiagrammi abil saate kiiresti visualiseerida seda kordajat. Diagramm, kus andmepunktid on korrektselt joondatud, näitab kõrget korrelatsiooni, kuid diagramm, kus andmepunktid on X ja Y vahel juhuslikult hajutatud, näitab madalat korrelatsiooni.

Hea lineaarse regressiooni mudel on selline, millel on kõrge (lähemal 1-le kui 0-le) korrelatsioonikordaja, kasutades väikseimate ruutude regressiooni meetodit koos regressioonijoonega.


## **2. Tants andmetega: andmeraami loomine, mida kasutatakse modelleerimiseks**

<p >
   <img src="../../../../../../translated_images/et/janitor.e4a77dd3d3e6a32e.jpg"
   width="700"/>
   <figcaption>Kunstiteos: @allison_horst</figcaption>


<!--![Kunstiteos: \@allison_horst](../../../../../../translated_images/et/janitor.e4a77dd3d3e6a32e.jpg){width="700"}-->


Laadi vajalikud teegid ja andmestik. Muuda andmed andmeraamiks, mis sisaldab andmete alamhulka:

-   Vali ainult kõrvitsad, mille hind on määratud buššeli järgi

-   Muuda kuupäev kuuks

-   Arvuta hind kõrge ja madala hinna keskmisena

-   Muuda hind, et see kajastaks buššeli koguse järgi määratud hinda

> Käsitlesime neid samme [eelmises õppetükis](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

Seiklushimu vaimus uurime [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), mis pakub lihtsaid funktsioone määrdunud andmete uurimiseks ja puhastamiseks. Näiteks vaatame oma andmete veerunimesid:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Me saame paremini. Muudame need veerunimed `friendR`-iks, teisendades need [snake_case](https://en.wikipedia.org/wiki/Snake_case) konventsiooniks, kasutades `janitor::clean_names`. Selle funktsiooni kohta lisateabe saamiseks: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Palju korda tehtud 🧹! Nüüd tants andmetega, kasutades `dplyr`-i nagu eelmises tunnis! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Tubli töö!👌 Sul on nüüd puhas ja korrastatud andmestik, mille põhjal saad luua oma uue regressioonimudeli!

Kas sooviksid hajuvusdiagrammi?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Hajuvälja diagram tuletab meelde, et meil on kuude andmed ainult augustist detsembrini. Tõenäoliselt vajame rohkem andmeid, et teha järeldusi lineaarsel viisil.

Vaatame uuesti oma modelleerimisandmeid:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Mis siis, kui tahaksime ennustada kõrvitsa `hinda` lähtudes `linna` või `pakendi` veergudest, mis on tekstivormingus? Või veel lihtsamalt, kuidas saaksime leida korrelatsiooni (mis eeldab, et mõlemad sisendid on numbrilised) näiteks `pakendi` ja `hinna` vahel? 🤷🤷

Masinõppe mudelid töötavad kõige paremini numbriliste tunnustega, mitte tekstiväärtustega, seega tuleb tavaliselt kategoriseeritud tunnused teisendada numbrilisteks esitusteks.

See tähendab, et peame leidma viisi, kuidas oma ennustajaid ümber vormindada, et mudel saaks neid tõhusamalt kasutada – seda protsessi nimetatakse `tunnuste insenerimiseks`.


## 3. Andmete ettevalmistamine modelleerimiseks retseptidega 👩‍🍳👨‍🍳

Tegevusi, mis muudavad ennustajate väärtused mudeli jaoks tõhusamaks kasutamiseks, nimetatakse `tunnuste insenerimiseks`.

Erinevatel mudelitel on erinevad eeltöötluse nõuded. Näiteks vajab vähimruutude meetod `kategooriliste muutujate kodeerimist`, nagu kuu, sort ja city_name. See tähendab lihtsalt `kategooriliste väärtustega` veeru `tõlkimist` üheks või mitmeks `numbriliseks veeruks`, mis asendavad algse veeru.

Näiteks, kui teie andmetes on järgmine kategooriline tunnus:

|  linn   |
|:-------:|
| Denver  |
| Nairobi |
|  Tokyo  |

Võite rakendada *järjestikust kodeerimist*, et asendada iga kategooria unikaalse täisarvuga, näiteks nii:

| linn |
|:----:|
|  0   |
|  1   |
|  2   |

Ja just seda me oma andmetega teeme!

Selles osas uurime veel üht suurepärast Tidymodels paketti: [recipes](https://tidymodels.github.io/recipes/) - mis on loodud selleks, et aidata teil oma andmeid **enne** mudeli treenimist eeltöödelda. Retsept on sisuliselt objekt, mis määratleb, milliseid samme tuleb andmekogumile rakendada, et see modelleerimiseks valmis saada.

Nüüd loome retsepti, mis valmistab meie andmed modelleerimiseks ette, asendades ennustajate veergudes kõik vaatlused unikaalse täisarvuga:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Vinge! 👏 Me just lõime oma esimese retsepti, mis määratleb tulemuse (hinna) ja selle vastavad ennustajad ning et kõik ennustajate veerud tuleks kodeerida täisarvudeks 🙌! Vaatame selle kiiresti üle:

- `recipe()` funktsiooni kasutamine koos valemiga määrab retseptile muutujate *rollid*, kasutades `new_pumpkins` andmeid viitena. Näiteks on `price` veerule määratud `outcome` roll, samas kui ülejäänud veerud on määratud `predictor` rolliks.

- `step_integer(all_predictors(), zero_based = TRUE)` määrab, et kõik ennustajad tuleks teisendada täisarvudeks, alustades nummerdamist nullist.

Me usume, et sul võib tekkida mõtteid nagu: "See on nii lahe!! Aga mis siis, kui ma tahaksin kinnitada, et retseptid teevad täpselt seda, mida ma ootan? 🤔"

See on suurepärane mõte! Vaata, kui retsept on määratletud, saad hinnata vajalikke parameetreid andmete eeltöötlemiseks ja seejärel andmed välja võtta. Tavaliselt ei ole seda vaja teha, kui kasutad Tidymodels'i (näeme peagi tavapärast lähenemist -> `workflows`), kuid see võib olla kasulik, kui tahad teha mingi kontrolli, et kinnitada, et retseptid toimivad ootuspäraselt.

Selleks vajad kahte lisakäsku: `prep()` ja `bake()`, ning nagu alati, aitavad meie väikesed R-sõbrad [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) sul sellest paremini aru saada!

<p >
   <img src="../../../../../../translated_images/et/recipes.9ad10d8a4056bf89.png"
   width="550"/>
   <figcaption>Illustratsioon autorilt @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): hindab vajalikke parameetreid treeningandmestiku põhjal, mida saab hiljem rakendada teistele andmestikele. Näiteks, antud ennustaja veeru puhul, millisele vaatlusele määratakse täisarv 0, 1, 2 jne.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): rakendab ettevalmistatud retsepti toimingud ükskõik millisele andmestikule.

Seega, valmistame ette ja rakendame oma retseptid, et tõesti kinnitada, et sisemiselt kodeeritakse ennustaja veerud kõigepealt enne, kui mudel sobitatakse.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Hurraa! 🥳 Töödeldud andmed `baked_pumpkins` sisaldavad kõiki kodeeritud ennustajaid, mis kinnitab, et meie retseptina määratletud eeltöötlusetapid toimivad ootuspäraselt. See muudab andmete lugemise keerulisemaks, kuid Tidymodelsile palju arusaadavamaks! Võta aega, et välja selgitada, milline vaatluse väärtus on vastavaks täisarvuks kaardistatud.

Samuti tasub mainida, et `baked_pumpkins` on andmeraamistik, millel saame teha arvutusi.

Näiteks proovime leida hea korrelatsiooni kahe andmepunkti vahel, et potentsiaalselt luua hea ennustav mudel. Selleks kasutame funktsiooni `cor()`. Kirjuta `?cor()`, et saada funktsiooni kohta rohkem teavet.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Selgub, et linna ja hinna vahel on ainult nõrk korrelatsioon. Kuid pakendi ja selle hinna vahel on veidi parem korrelatsioon. See on loogiline, eks? Tavaliselt, mida suurem on tootepakend, seda kõrgem on hind.

Kui me juba teemaga tegeleme, proovime visualiseerida ka kõigi veergude korrelatsioonimaatriksit, kasutades `corrplot` paketti.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Palju parem.

Hea küsimus, mida nüüd selle andmestiku kohta küsida, oleks: '`Millist hinda võin oodata konkreetse kõrvitsapaketi eest?`' Sukeldume kohe teemasse!

> Märkus: Kui **`küpsetate()`** ettevalmistatud retsepti **`pumpkins_prep`** koos **`new_data = NULL`**, siis saate töödeldud (st kodeeritud) treeningandmed. Kui teil oleks näiteks teine andmestik, näiteks testandmestik, ja sooviksite näha, kuidas retsept seda eeltöötleks, küpsetaksite lihtsalt **`pumpkins_prep`** koos **`new_data = test_set`**

## 4. Lineaarse regressioonimudeli loomine

<p >
   <img src="../../../../../../translated_images/et/linear-polynomial.5523c7cb6576ccab.png"
   width="800"/>
   <figcaption>Infograafik: Dasani Madipalli</figcaption>


<!--![Infograafik: Dasani Madipalli](../../../../../../translated_images/et/linear-polynomial.5523c7cb6576ccab.png){width="800"}-->


Nüüd, kui oleme retsepti loonud ja kinnitanud, et andmed töödeldakse õigesti, ehitame regressioonimudeli, et vastata küsimusele: `Millist hinda võin oodata antud kõrvitsapakendi eest?`

#### Treeni lineaarse regressiooni mudel treeningandmestiku abil

Nagu te ilmselt juba aru saite, on veerg *price* `tulemusmuutuja`, samas kui veerg *package* on `ennustaja` muutuja.

Selleks jagame esmalt andmed nii, et 80% läheb treeningandmestikku ja 20% testandmestikku, seejärel määratleme retsepti, mis kodeerib ennustaja veeru täisarvudeks, ja seejärel loome mudeli spetsifikatsiooni. Me ei valmista ega küpseta oma retsepti, kuna teame juba, et see töötleb andmeid ootuspäraselt.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Tubli töö! Nüüd, kui meil on olemas retsept ja mudeli spetsifikatsioon, peame leidma viisi, kuidas need kokku siduda üheks objektiks, mis esmalt eeltöötleb andmed (kulisside taga toimub prep+bake), sobitab mudeli eeltöödeldud andmetega ja võimaldab ka võimalikke järeltöötluse tegevusi. Kuidas see kõlab sinu meelerahu jaoks!🤩

Tidymodelsis nimetatakse seda mugavat objekti [`workflow`](https://workflows.tidymodels.org/) ja see hoiab mugavalt sinu modelleerimise komponente! Seda võiksime *Pythonis* nimetada *torujuhtmeteks* (*pipelines*).

Nii et paneme kõik kokku ühte workflow'sse!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 Peale selle saab töövoogu sobitada/treenida peaaegu samamoodi nagu mudelit.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Mudeli väljundist näeme treeningu käigus õpitud koefitsiente. Need esindavad parima sobivuse joone koefitsiente, mis annavad meile tegeliku ja prognoositud muutuja vahelise väikseima üldise vea.

#### Mudeli jõudluse hindamine testandmestiku abil

On aeg näha, kuidas mudel toimis 📏! Kuidas me seda teeme?

Nüüd, kui oleme mudeli treeninud, saame seda kasutada prognooside tegemiseks test_andmestiku jaoks, kasutades `parsnip::predict()`. Seejärel saame neid prognoose võrrelda tegelike märgiste väärtustega, et hinnata, kui hästi (või mitte!) mudel töötab.

Alustame prognooside tegemisest testandmestiku jaoks ja seejärel ühendame veerud testandmestikuga.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Jah, sa oled just mudeli treeninud ja kasutanud seda ennustuste tegemiseks!🔮 Kas see on hea? Vaatame mudeli jõudlust!

Tidymodelsis teeme seda `yardstick::metrics()` abil! Lineaarse regressiooni puhul keskendume järgmistele mõõdikutele:

-   `Root Mean Square Error (RMSE)`: [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) ruutjuur. See annab absoluutse mõõdiku samas ühikus kui silt (antud juhul kõrvitsa hind). Mida väiksem väärtus, seda parem mudel (lihtsustatult öeldes esindab see keskmist hinda, mille võrra ennustused eksivad!)

-   `Determineerimistegur (tavaliselt tuntud kui R-ruut või R2)`: Suhteline mõõdik, mille puhul suurem väärtus tähendab paremat mudeli sobivust. Sisuliselt esindab see mõõdik seda, kui palju mudel suudab selgitada ennustatud ja tegelike siltide väärtuste vahelise variatsiooni.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Seal läks mudeli jõudlus. Vaatame, kas saame parema ülevaate, visualiseerides hajuvusdiagrammi, kus on pakett ja hind, ning kasutame tehtud ennustusi, et lisada parima sobivuse joon.

See tähendab, et peame testkomplekti ette valmistama ja töötlema, et kodeerida paketi veerg ning seejärel siduda see meie mudeli tehtud ennustustega.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Nagu näha, ei suuda lineaarne regressioonimudel eriti hästi üldistada seost paki ja selle vastava hinna vahel.

🎃 Palju õnne, sa just lõid mudeli, mis aitab ennustada mõne kõrvitsasordi hinda. Sinu pühade kõrvitsapõld saab olema imeline. Kuid tõenäoliselt suudad luua veelgi parema mudeli!

## 5. Loo polünoomse regressiooni mudel

<p >
   <img src="../../../../../../translated_images/et/linear-polynomial.5523c7cb6576ccab.png"
   width="800"/>
   <figcaption>Infograafik: Dasani Madipalli</figcaption>


Mõnikord ei pruugi meie andmetel olla lineaarset seost, kuid me soovime siiski prognoosida tulemusi. Polünoomiline regressioon aitab meil teha prognoose keerukamate mittelineaarsete seoste korral.

Võtame näiteks seose pakendi ja hinna vahel meie kõrvitsate andmestikus. Kuigi mõnikord on muutujate vahel lineaarne seos – mida suurem on kõrvitsa maht, seda kõrgem hind –, ei saa neid seoseid alati kujutada tasapinnana või sirgjoonena.

> ✅ Siin on [veel mõned näited](https://online.stat.psu.edu/stat501/lesson/9/9.8) andmetest, mille puhul võiks kasutada polünoomilist regressiooni
>
> Vaata uuesti seost sordi ja hinna vahel eelmises graafikus. Kas see hajusdiagramm tundub, et seda peaks tingimata analüüsima sirgjoonega? Võib-olla mitte. Sellisel juhul võid proovida polünoomilist regressiooni.
>
> ✅ Polünoomid on matemaatilised avaldised, mis võivad koosneda ühest või mitmest muutujast ja kordajatest.

#### Treeni polünoomilise regressiooni mudelit treeningandmestiku abil

Polünoomiline regressioon loob *kõvera joone*, et paremini sobitada mittelineaarseid andmeid.

Vaatame, kas polünoomiline mudel suudab prognoosides paremini toimida. Järgime sarnast protseduuri nagu varem:

-   Loo retsept, mis määrab kindlaks andmete eeltöötluse sammud, et need modelleerimiseks valmis saada, näiteks: ennustajate kodeerimine ja polünoomide arvutamine astmega *n*

-   Koosta mudeli spetsifikatsioon

-   Koonda retsept ja mudeli spetsifikatsioon töövoogu

-   Loo mudel, sobitades töövoogu

-   Hinda, kui hästi mudel testandmetel toimib

Asume asja kallale!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Hinda mudeli jõudlust

👏👏Oled loonud polünoommudeli, teeme nüüd prognoose testandmestikul!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, hindame, kuidas mudel testikomplektil toimis, kasutades `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Palju parem jõudlus.

`rmse` vähenes umbes 7-lt umbes 3-le, mis viitab väiksemale veale tegeliku hinna ja prognoositud hinna vahel. Seda võib *umbes* tõlgendada nii, et keskmiselt on valed prognoosid ekslikud umbes 3 dollari võrra. `rsq` suurenes umbes 0.4-lt 0.8-le.

Kõik need mõõdikud näitavad, et polünoomiline mudel toimib palju paremini kui lineaarne mudel. Tubli töö!

Vaatame, kas saame seda visualiseerida!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Näete kõverjoont, mis sobib teie andmetega paremini! 🤩

Saate selle veelgi sujuvamaks muuta, kui edastate polünoomvalemi `geom_smooth`-ile, näiteks nii:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Nagu sujuv kõver!🤩

Siin on, kuidas saaksid teha uue ennustuse:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


`polynomial model` ennustus tundub mõistlik, arvestades `price` ja `package` hajuvusdiagramme! Ja kui see mudel on parem kui eelmine, vaadates samu andmeid, siis tuleb eelarvesse lisada need kallimad kõrvitsad!

🏆 Tubli töö! Sa lõid ühe tunni jooksul kaks regressioonimudelit. Regressiooni viimases osas õpid logistilist regressiooni, et määrata kategooriaid.

## **🚀Väljakutse**

Katseta mitmeid erinevaid muutujaid selles märkmikus, et näha, kuidas korrelatsioon vastab mudeli täpsusele.

## [**Loengu järgne viktoriin**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Ülevaade ja iseseisev õppimine**

Selles tunnis õppisime lineaarset regressiooni. On ka teisi olulisi regressioonitüüpe. Loe Stepwise, Ridge, Lasso ja Elasticnet tehnikate kohta. Hea kursus, mida õppida, on [Stanfordi statistilise õppimise kursus](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Kui soovid rohkem teada saada, kuidas kasutada suurepärast Tidymodels raamistikku, vaata järgmisi ressursse:

-   Tidymodels veebisait: [Alusta Tidymodelsiga](https://www.tidymodels.org/start/)

-   Max Kuhn ja Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **TÄNUD:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) suurepäraste illustratsioonide loomise eest, mis muudavad R-i kasutamise sõbralikumaks ja kaasahaaravamaks. Leia rohkem illustratsioone tema [galeriist](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Lahtiütlus**:  
See dokument on tõlgitud, kasutades AI tõlketeenust [Co-op Translator](https://github.com/Azure/co-op-translator). Kuigi püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algkeeles tuleks lugeda autoriteetseks allikaks. Olulise teabe puhul on soovitatav kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valede tõlgenduste eest.
